In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, BasicAer, execute
from qiskit.aqua.utils.circuit_factory import CircuitFactory
from exp_a4 import A4Factory
from exp_a23 import A23Factory
from exp_a1 import A1Factory
import numpy as np
from qiskit.tools.visualization import circuit_drawer
from scipy.linalg import expm
%matplotlib inline
from qiskit.quantum_info.operators import Operator
from qiskit.aqua.circuits.gates import mct
from qiskit import IBMQ
from qiskit import BasicAer
# import FTSDFactory

In [2]:
n=2
t=1
m_list = []
m = 11 #The order of accuracy
m_list.append(m)

#Library for k_vec (given in arxiv:1907.11679)
kvec2 = [1, 2]
kvec3 = [1, 2, 6]
kvec4 = [1, 2, 3, 10]
kvec5 = [1, 2, 3, 5, 17]
kvec6 = [1, 2, 3, 4, 6, 21]
kvec7 = [1, 2, 3, 4, 5, 9, 34]
kvec8 = [1, 2, 3, 4, 5, 6, 12, 45]
kvec9 = [1, 2, 3, 4, 5, 6, 8, 15, 58]
kvec10 = [1, 2, 3, 4, 5, 6, 7, 10, 18, 72]
kvec11 = [1, 2, 3, 4, 5, 6, 7, 8, 12, 22, 88]
kvec12 = [1, 2, 3, 4, 5, 6, 7, 8, 10, 14, 27, 106]
kvec13 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 16, 31, 121]
kvec14 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 19, 37, 147]
kvec15 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 22, 42, 170]


#     m = m_input
k_vec = kvec11
#for i in range(m):
#   k_vec.append(1+i)
#print(k_vec)
n_sum = int(np.ceil(np.log2(len(k_vec))))

# number of ancilla qubits for the unitary operations themselves
n_ancilla = n-1

# number of ancilla qubits for controlling the unitary operations
n_a_con = 1

# number of ancilla qubits for multi-controlled toffoli
n_toff = n_sum-2

# Hamiltonian setting
matrix_index = [0, 1, 2]  # 0:a1, 1:a23, 2:a4
coefficient = [np.pi/4, np.pi/4, np.pi/4]  # params(e.g. theta) of each matrix

In [3]:
#Calculating Matrix directly for first order Trotter Suzuki LCU
initial_vector = np.append(1,[0]*((2**n)-1))
M = len(coefficient)

for j in range(len(k_vec)):
    kj = k_vec[j]
    for l in range(kj):
        for i in range(len(coefficient)):
            if matrix_index[i] == 0:
                b_1 = coefficient[i]*t/kj/M          
                matrixa1 = np.diag(np.array([b_1]*(2**n)))
                expmat1 = expm(-1j*matrixa1*t)
                nowstate = np.dot(expmat1, initial_vector)
                initial_vector = nowstate
            elif matrix_index[i]==1:
                b_2 = coefficient[i]*t/kj/M
                matrixa23 = np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), -1) + np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), 1)
                expmat23 = expm(-1j*matrixa23*t)
                nowstate = np.dot(expmat23, initial_vector)
                initial_vector = nowstate
            elif matrix_index[i]==2:
                b_4 = coefficient[i]*t/kj/M
                matrixa4 =np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), -1) + np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), 1)
                expmat4 = expm(-1j*matrixa4*t)
                nowstate = np.dot(expmat4, initial_vector)
                initial_vector = nowstate
            else:
                exit("Wrong matrix index: ", i)   

#print(matrixa1)
#print(matrixa23)
#print(matrixa4)
np.set_printoptions(threshold=100000, linewidth=1000)
#print(expmat1)
#print(expmat23)
#print(expmat4)
print(initial_vector)
circuit_simulator1 = initial_vector

[ 0.18312231+0.04906747j  0.00947652-0.03536686j -0.04856253-0.01301229j  0.25361498-0.94650398j]


In [4]:
#Calculating Matrix directly for second order Trotter Suzuki LCU (if you make DU1=False in the FTDS_Factory)
initial_vector = np.append(1,[0]*((2**n)-1))
M = len(coefficient)

for j in range(len(k_vec)):
    kj = k_vec[j]
    for l in range(kj):
        for i in range(len(matrix_index)):
            if matrix_index[i]==0:
                b_1 = coefficient[i]*t/kj/M          
                matrixa1 = np.diag(np.array([b_1]*(2**n)))
                expmat1 = expm(-1j*matrixa1*t)
                nowstate = np.dot(expmat1, initial_vector)
                initial_vector = nowstate
            elif matrix_index[i]==1:
                b_2 = coefficient[i]*t/kj/M
                matrixa23 = np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), -1) + np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), 1)
                expmat23 = expm(-1j*matrixa23*t)
                nowstate = np.dot(expmat23, initial_vector)
                initial_vector = nowstate
            elif matrix_index[i]==2:
                b_4 = coefficient[i]*t/kj/M
                matrixa4 =np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), -1) + np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), 1)
                expmat4 = expm(-1j*matrixa4*t)
                nowstate = np.dot(expmat4, initial_vector)
                initial_vector = nowstate
            else:
                exit("Wrong matrix index: ", i)
        for i in reversed(range(len(matrix_index))):
            if matrix_index[i]==0:
                b_1 = coefficient[i]*t/kj/M          
                matrixa1 = np.diag(np.array([b_1]*(2**n)))
                expmat1 = expm(-1j*matrixa1*t)
                nowstate = np.dot(expmat1, initial_vector)
                initial_vector = nowstate
            elif matrix_index[i]==1:
                b_2 = coefficient[i]*t/kj/M
                matrixa23 = np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), -1) + np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), 1)
                expmat23 = expm(-1j*matrixa23*t)
                nowstate = np.dot(expmat23, initial_vector)
                initial_vector = nowstate
            elif matrix_index[i]==2:
                b_4 = coefficient[i]*t/kj/M
                matrixa4 =np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), -1) + np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), 1)
                expmat4 = expm(-1j*matrixa4*t)
                nowstate = np.dot(expmat4, initial_vector)
                initial_vector = nowstate
            else:
                exit("Wrong matrix index: ", i)

#print(matrixa1)
#print(matrixa23)
#print(matrixa4)
np.set_printoptions(threshold=100000, linewidth=1000)
#print(expmat1)
#print(expmat23)
#print(expmat4)
print(initial_vector)
circuit_simulator2 = initial_vector
np.linalg.norm(circuit_simulator2, ord=2)

[-0.81612411-0.47118948j -0.06549788+0.11344566j -0.0438812 -0.02533482j  0.15181783-0.26295619j]


1.000000000000021

In [5]:
initial_vector = np.append(1, [0]*((2**n)-1))

H_sum = np.zeros(shape=[2**n, 2**n], dtype='float32')
for i in range(len(matrix_index)):
    if matrix_index[i] == 0:
        b_1 = coefficient[i]*t            
        H_sum += np.diag(np.array([b_1]*(2**n)))
    elif matrix_index[i]==1:
        b_2 = coefficient[i]*t
        H_sum += np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), -1) + np.diag(np.append(np.array([0, b_2]*((2**(n-1))-1)),0), 1)
    elif matrix_index[i]==2:
        b_4 = coefficient[i]*t
        H_sum += np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), -1) + np.diag(np.append(np.array([b_4, 0]*((2**(n-1))-1)),b_4), 1)
    else:
        exit("Wrong matrix index: ", i)
#print(H_sum)
exact_exp = expm(-1j * H_sum * t * 1.0)

ret = np.dot(exact_exp, initial_vector)
#ret /= np.linalg.norm(ret) the norm should be one
print(ret)
np.linalg.norm(ret, ord=2)

[ 0.51031858-0.51031852j -0.44964504-0.44964504j -0.18624906+0.18624908j  0.0520151 +0.0520151j ]


0.9999999708970568

In [6]:
Error = np.linalg.norm(circuit_simulator1-ret, ord=2)
print(Error)

1.3779458122096597


In [7]:
Error = np.linalg.norm(circuit_simulator2-ret, ord=2)
print(Error)

1.549136960063805


In [8]:
1.348576678891715

1.348576678891715